In [ ]:
!pip uninstall -y diffusers transformers huggingface_hub peft accelerate
!pip install torch torchvision --index-url https://download.pytorch.org/whl/cu121


Found existing installation: diffusers 0.27.0
Uninstalling diffusers-0.27.0:
  Successfully uninstalled diffusers-0.27.0
Found existing installation: transformers 4.57.1
Uninstalling transformers-4.57.1:
  Successfully uninstalled transformers-4.57.1
Found existing installation: huggingface-hub 0.25.2
Uninstalling huggingface-hub-0.25.2:
  Successfully uninstalled huggingface-hub-0.25.2
Found existing installation: peft 0.17.1
Uninstalling peft-0.17.1:
  Successfully uninstalled peft-0.17.1
Found existing installation: accelerate 1.11.0
Uninstalling accelerate-1.11.0:
  Successfully uninstalled accelerate-1.11.0
Looking in indexes: https://download.pytorch.org/whl/cu121


In [ ]:
# Colab python cell
from getpass import getpass
import os
hf_token = getpass("Paste your Hugging Face token (or press Enter if not needed): ")
if hf_token:
    os.environ["HF_TOKEN"] = hf_token


Paste your Hugging Face token (or press Enter if not needed): ··········


In [ ]:
!pip install "diffusers==0.29.0" "transformers==4.46.2" "accelerate==0.33.0" \
             "huggingface_hub==0.25.2" "peft==0.10.0" "safetensors" "scipy" "ftfy"


In [ ]:
import torch
from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler
from PIL import Image
import time, os
print("✅ Imports OK")


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

✅ Imports OK


/usr/local/lib/python3.12/dist-packages/diffusers/models/transformers/transformer_2d.py:34: FutureWarning: `Transformer2DModelOutput` is deprecated and will be removed in version 1.0.0. Importing `Transformer2DModelOutput` from `diffusers.models.transformer_2d` is deprecated and this will be removed in a future version. Please use `from diffusers.models.modeling_outputs import Transformer2DModelOutput`, instead.
  deprecate("Transformer2DModelOutput", "1.0.0", deprecation_message)


In [ ]:
# Example: Check Stable Diffusion model by generating Floorplan & Moodboard samples

import torch, time, os
from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler
from PIL import Image

# -------------------------
# Setup
# -------------------------
model_id = "stabilityai/stable-diffusion-2-1"
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"🔧 Loading model '{model_id}' on {device}...")

pipe = StableDiffusionPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    safety_checker=None
)
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
pipe = pipe.to(device)
pipe.enable_attention_slicing()

print("✅ Model loaded successfully.")

# -------------------------
# 1️⃣ Floorplan Example
# -------------------------
floorplan_prompt = (
    "modern 3BHK apartment 2D floor plan, labeled rooms, top-down architectural blueprint, "
    "white background, clean black vector lines, technical drawing, 4x5 living room, 3x3 kitchen, 4x4 bedroom"
)

print("\n🏗️ Generating floorplan sample...")
start = time.time()
floor_image = pipe(floorplan_prompt, num_inference_steps=40, guidance_scale=9.0).images[0]
floor_image.save("sample_floorplan.png")
print(f"✅ Floorplan saved as sample_floorplan.png ({time.time()-start:.1f}s)")

# -------------------------
# 2️⃣ Moodboard Example
# -------------------------
moodboard_prompt = (
    "modern bedroom interior design moodboard collage, textures, olive green fabric, "
    "brass lamp, rattan furniture, minimalist warm lighting, 8k photorealistic aesthetic"
)

print("\n🎨 Generating moodboard sample...")
start = time.time()
mood_image = pipe(moodboard_prompt, num_inference_steps=35, guidance_scale=8.5).images[0]
mood_image.save("sample_moodboard.png")
print(f"✅ Moodboard saved as sample_moodboard.png ({time.time()-start:.1f}s)")

# -------------------------
# Show results
# -------------------------
floor_image.show()
mood_image.show()

print("\n✨ Test complete! Check 'sample_floorplan.png' and 'sample_moodboard.png' in your directory.")


🔧 Loading model 'stabilityai/stable-diffusion-2-1' on cuda...


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

✅ Model loaded successfully.

🏗️ Generating floorplan sample...


  0%|          | 0/40 [00:00<?, ?it/s]

✅ Floorplan saved as sample_floorplan.png (33.3s)

🎨 Generating moodboard sample...


  0%|          | 0/35 [00:00<?, ?it/s]

✅ Moodboard saved as sample_moodboard.png (29.9s)

✨ Test complete! Check 'sample_floorplan.png' and 'sample_moodboard.png' in your directory.


In [ ]:
# app.py
"""
GenAI Design Assistant — Stable Diffusion 2.1 (Crisp Text Edition)
Auto Type Detection + Pillow Overlay for Sharp Labels & Dimensions
"""

import os, re, torch, gradio as gr
from PIL import Image, ImageEnhance, ImageFilter, ImageDraw, ImageFont
from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler

# -------------------------  MODEL  -------------------------
MODEL_ID = os.environ.get("MODEL_ID", "stabilityai/stable-diffusion-2-1")
HF_TOKEN = os.environ.get("HF_TOKEN", None)

def load_pipe():
    auth = {"use_auth_token": HF_TOKEN} if HF_TOKEN else {}
    dtype = torch.float16 if torch.cuda.is_available() else torch.float32
    pipe = StableDiffusionPipeline.from_pretrained(
        MODEL_ID, torch_dtype=dtype, safety_checker=None, **auth
    )
    pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
    pipe.to("cuda" if torch.cuda.is_available() else "cpu")
    pipe.enable_attention_slicing()
    return pipe

pipe = load_pipe()

# -------------------------  HELPERS  -------------------------
NEGATIVE_PROMPT = (
    "blurry, low quality, distorted, gibberish text, watermark, duplicate, noise, "
    "crooked lines, wrong perspective, extra walls, unrealistic proportions"
)

def detect_image_type(prompt: str):
    floor = ["floor", "layout", "blueprint", "plan", "bhk", "villa", "map", "room size"]
    mood = ["interior", "decor", "sofa", "lamp", "texture", "wall", "mood", "bedroom"]
    p = prompt.lower()
    if any(k in p for k in floor): return "Floorplan"
    if any(k in p for k in mood): return "Moodboard"
    return "Floorplan"

def refine_prompt(prompt: str, kind: str):
    p = prompt.strip().capitalize()
    if kind == "Floorplan":
        return (
            f"{p}, professional architectural 2D top-down floor plan, "
            "technical drawing, clean black lines, white background, "
            "precise proportions, schematic layout, no blur, no text"
        )
    return (
        f"{p}, high-end interior design moodboard collage, photorealistic, "
        "textures, furniture, warm lighting, 8k render, aesthetic composition"
    )

def extract_labels_dims(text: str):
    text = text.replace("×", "x").replace("X", "x")
    pairs = re.findall(r"([A-Za-z0-9_ ]{2,30})\s+(\d+\.?\d*\s*x\s*\d+\.?\d*)", text)
    labels, dims = [], []
    for l, d in pairs:
        labels.append(l.strip())
        dims.append(d.replace(" ", ""))
    return labels, dims

# -------------------------  POSTPROCESS  -------------------------
def load_font(size=32):
    try:
        return ImageFont.truetype("DejaVuSansMono-Bold.ttf", size)
    except Exception:
        return ImageFont.load_default()

def overlay_text(image: Image.Image, labels, dims):
    """Add ultra-sharp text labels/dimensions evenly spaced on image."""
    draw = ImageDraw.Draw(image)
    w, h = image.size
    n = max(len(labels), len(dims))
    step_y = max(80, h // (n + 2))
    x_left, x_right = int(w * 0.05), int(w * 0.55)
    font = load_font(28)
    for i in range(n):
        y = 50 + i * step_y
        label = labels[i] if i < len(labels) else ""
        dim = dims[i] if i < len(dims) else ""
        # Text with white stroke for clarity
        draw.text((x_left, y), f"{label}", fill=(0,0,0),
                  font=font, stroke_width=3, stroke_fill=(255,255,255))
        draw.text((x_right, y), f"{dim}", fill=(30,30,30),
                  font=font, stroke_width=2, stroke_fill=(255,255,255))
    return image

def enhance_image(image: Image.Image):
    image = image.filter(ImageFilter.UnsharpMask(radius=2, percent=180, threshold=2))
    image = ImageEnhance.Sharpness(image).enhance(1.6)
    image = ImageEnhance.Contrast(image).enhance(1.15)
    image = ImageEnhance.Brightness(image).enhance(1.03)
    return image

# -------------------------  GENERATION  -------------------------
def generate(prompt, steps, scale, seed, aspect, show_text):
    kind = detect_image_type(prompt)
    refined = refine_prompt(prompt, kind)
    labels, dims = extract_labels_dims(prompt)
    width, height = (1024,768) if aspect=="Wide" else (768,1024) if aspect=="Tall" else (896,896)
    device = "cuda" if torch.cuda.is_available() else "cpu"
    gen = torch.Generator(device=device).manual_seed(int(seed))
    result = pipe(prompt=refined, negative_prompt=NEGATIVE_PROMPT,
                  num_inference_steps=int(steps), guidance_scale=float(scale),
                  generator=gen, width=width, height=height)
    img = result.images[0].convert("RGB")
    img = enhance_image(img)
    if show_text and kind == "Floorplan" and (labels or dims):
        img = overlay_text(img, labels, dims)
    img = enhance_image(img)
    save = f"output_{kind.lower()}_{seed}.png"
    img.save(save)
    return img, save, kind

# -------------------------  UI  -------------------------
examples = [
    ["Modern 3BHK apartment living 4x5, kitchen 3x3, bed1 4x4", 40, 9.0, 42, "Wide", True],
    ["Scandinavian bedroom mood board with oak wood textures and white linen", 35, 8.5, 77, "Square", False],
]

css = """
body { background: linear-gradient(135deg,#edf2fb,#e3ebf9); }
.gradio-container { font-family:'Segoe UI',Roboto,sans-serif; }
h2 { color:#111;font-weight:600; }
"""

with gr.Blocks(title="Thanush GEN AI IMAGE", css=css) as demo:
    gr.Markdown("<h2>Thanush GenAI Design Assistant — Crisp Text Edition</h2>")
    gr.Markdown("Smartly generates floorplans or moodboards with perfectly readable dimensions and labels.")

    with gr.Row():
        with gr.Column(scale=3):
            prompt_in = gr.Textbox(lines=3, label="📝 Design Prompt",
                placeholder="e.g., 3BHK apartment layout: living 4x5, kitchen 3x3, bed1 4x4")
            aspect = gr.Radio(["Wide","Square","Tall"], value="Wide", label="Aspect Ratio")
            show_text = gr.Checkbox(True, label="Add Sharp Labels & Dimensions")
            steps = gr.Slider(20,75,40,1,label="Inference Steps")
            scale = gr.Slider(5,12,9.0,0.5,label="Guidance Scale")
            seed = gr.Number(42, label="Seed")
            btn = gr.Button("✨ Generate Crisp Image", variant="primary")
            gr.Examples(examples=examples, inputs=[prompt_in, steps, scale, seed, aspect, show_text])
        with gr.Column(scale=4):
            out_img = gr.Image(type="pil", label="Generated Image")
            out_kind = gr.Textbox(label="🧭 Detected Type", interactive=False)
            out_file = gr.File(label="Download PNG")

    btn.click(fn=generate, inputs=[prompt_in, steps, scale, seed, aspect, show_text],
              outputs=[out_img, out_file, out_kind])

if __name__ == "__main__":
    demo.launch(server_name="0.0.0.0", share=True)


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://2675514e9496685501.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
